In [1]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Parquet Analysis") \
    .getOrCreate()

# Load Parquet dataset
df = spark.read.parquet("parquet_database")

# Show schema to verify structure
df.printSchema()


24/12/11 06:49:39 WARN Utils: Your hostname, Abhinavs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.16 instead (on interface en0)
24/12/11 06:49:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/11 06:49:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/12/11 06:49:43 WARN DataSource: [COLUMN_ALREADY_EXISTS] The column `receivedate` already exists. Consider to choose another name or rename the existing column.


root
 |-- safetyreportid: string (nullable = true)
 |-- transmissiondateformat: string (nullable = true)
 |-- transmissiondate: string (nullable = true)
 |-- serious: string (nullable = true)
 |-- receivedateformat: string (nullable = true)
 |-- receiptdateformat: string (nullable = true)
 |-- receiptdate: string (nullable = true)
 |-- fulfillexpeditecriteria: string (nullable = true)
 |-- companynumb: string (nullable = true)
 |-- receiver: double (nullable = true)
 |-- primarysource.reportercountry: string (nullable = true)
 |-- primarysource.qualification: string (nullable = true)
 |-- sender.senderorganization: string (nullable = true)
 |-- patient.reaction: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- reactionmeddrapt: string (nullable = true)
 |    |    |-- reactionmeddraversionpt: string (nullable = true)
 |-- patient.drug: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- activesubstance: struct (nul

In [2]:
from pyspark.sql.functions import col, explode, year, month, concat_ws
import matplotlib.pyplot as plt

# Aggregate by Year-Month
df = df.withColumn("YearMonth", concat_ws("-", year(col("receivedate")), month(col("receivedate"))))

# Explode `patient.reaction` and group by YearMonth
reaction_trends = (
    df.select(col("YearMonth"), explode(col("`patient.reaction`")).alias("reaction"))
    .groupBy("YearMonth", "reaction.reactionmeddrapt")  # Group by YearMonth and reaction
    .count()
    .orderBy("YearMonth")
)

# Convert to Pandas for further analysis
reaction_trends_pd = reaction_trends.toPandas()

# Focus on top N reactions globally
top_reactions = (
    reaction_trends_pd.groupby("reactionmeddrapt")["count"].sum().nlargest(5).index
)
reaction_trends_filtered = reaction_trends_pd[
    reaction_trends_pd["reactionmeddrapt"].isin(top_reactions)
]

# Pivot for visualization
reaction_pivot = reaction_trends_filtered.pivot(
    index="YearMonth", columns="reactionmeddrapt", values="count"
).fillna(0)

# Plot top reactions
plt.figure(figsize=(14, 7))
for reaction in reaction_pivot.columns:
    plt.plot(reaction_pivot.index, reaction_pivot[reaction], label=reaction)

# Customize the plot
plt.xlabel("Year-Month")
plt.ylabel("Count")
plt.title("Reaction Trends Over Time (Aggregated by Year-Month)")
plt.legend(title="Reactions", loc='upper left', bbox_to_anchor=(1, 1))
plt.grid()
plt.xticks(rotation=45)

# Ensure layout is tight for better display
plt.tight_layout()
plt.show()


ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


#
# A fatal error has been detected by the Java Runtime Environment:
#
#  SIGSEGV (0xb) at pc=0x00000001038c511c, pid=79450, tid=71683
#
# JRE version: OpenJDK Runtime Environment Homebrew (11.0.25) (build 11.0.25+0)
# Java VM: OpenJDK 64-Bit Server VM Homebrew (11.0.25+0, mixed mode, tiered, compressed oops, g1 gc, bsd-aarch64)
# Problematic frame:
# V  [libjvm.dylib+0x69511c]  ObjectSynchronizer::inflate(Thread*, oopDesc*, ObjectSynchronizer::InflateCause)+0x18c
#
# No core dump will be written. Core dumps have been disabled. To enable core dumping, try "ulimit -c unlimited" before starting Java again
#
# An error report file with more information is saved as:
# /Volumes/abhinav/Project/hs_err_pid79450.log
[thread 71939 also had an error]
#
# If you would like to submit a bug report, please visit:
#   https://github.com/Homebrew/homebrew-core/issues
#


ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/socket.py", line 720, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
ConnectionResetError: [Errno 54] Connection reset by peer

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


ConnectionRefusedError: [Errno 61] Connection refused

In [ ]:
# 5. Average Age of Patients by Reaction


In [ ]:
from pyspark.sql.functions import col, explode, avg, when

# Normalize patient age based on patientonsetageunit
df = df.withColumn(
    "normalized_age",
    when(col("`patient.patientonsetageunit`") == "801", col("`patient.patientonsetage`").cast("double"))  # Years
    .when(col("`patient.patientonsetageunit`") == "802", col("`patient.patientonsetage`").cast("double") / 12)  # Months -> years
    .when(col("`patient.patientonsetageunit`") == "803", col("`patient.patientonsetage`").cast("double") / 365)  # Days -> years
    .otherwise(None)
)

# Calculate average age for each reaction type
avg_age_by_reaction = (
    df.select(
        explode(col("`patient.reaction`")).alias("reaction"),  # Explode reaction array
        col("normalized_age").alias("patientonsetage")
    )
    .groupBy("reaction.reactionmeddrapt")
    .agg(avg("patientonsetage").alias("avg_age"))
    .orderBy("avg_age", ascending=False)
    .limit(10)
)

# Convert to Pandas for visualization
avg_age_by_reaction_pd = avg_age_by_reaction.toPandas()

# Plot
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.barh(avg_age_by_reaction_pd["reactionmeddrapt"], avg_age_by_reaction_pd["avg_age"])
plt.xlabel("Average Age")
plt.ylabel("Reaction")
plt.title("Top 10 Reactions by Average Patient Age")
plt.gca().invert_yaxis()
plt.show()
